## From Scientific Questions to `Python Codes`
An example of workflow exploring global sea surface temperature data.
* Wenchang Yang (wenchang@princeton.edu)
* Department of Geosciences, Princeton University
* Junior Colloquium, Oct. 21, 2019

## What's covered so far
1. Python basics: `number`, `string`, `list`, `function`, `module`, `package`
2. Scientific computation: `numpy`, `scipy`
3. Data visualization: `matplotlib`, `cartopy`
4. **`xarray`**: high-level, user friendly, computing+plotting

Another very useful yet less touched Python package: `pandas`.

## What is `xarray` able to do? 
<img src="http://xarray.pydata.org/en/stable/_static/dataset-diagram-logo.png" width="300">

1. Open/save datasets (single/multiple, local/remote): `open_dataset`, `open_mfdataset`.
2. Data selection: `sel`, `isel`.
3. Computation: `mean`, `std`, `max`, `min`, `differentiate`, `integrate`, ...
4. Split-apply-combine: `groupby`.
4. Plot: `plot`, `plot.line`, `plot.contourf`, `plot.hist` ...

## What we do today
Apply `Python/xarray` to explore the global sea surface temperature (SST) variations.

## Data
* ERSST version 5: global **monthly** SST.
* $2^\circ$ longitude $\times$ $2^\circ$ latitude 
* It covers 1854-present, but we focus on **1979-2018** today.
* From Columbia University [data library](http://iridl.ldeo.columbia.edu): http://iridl.ldeo.columbia.edu/SOURCES/.NOAA/.NCDC/.ERSST/.version5/.sst/T/%28Jan%201979%29/%28Dec%202018%29/RANGE/X//lon/renameGRID/Y//lat/renameGRID/T//time/renameGRID/time/(days%20since%201979-01-01)/streamgridunitconvert%5Bzlev%5Daverage

## Scientific Questions
1. Is SST getting warmer over the past four decades? Everywhere?
2. How do El Nino/La Nina vary during this period?
    
![](https://blog.weatherops.com/hubfs/blog-files/elnino-vs-lanina-noaa.jpg)
https://blog.weatherops.com/hubfs/blog-files/elnino-vs-lanina-noaa.jpg

In [1]:
# xarray is the core package we are going to use
import xarray as xr
import matplotlib.pyplot as plt

In [2]:
%config InlineBackend.figure_format ='retina'
plt.rcParams['figure.dpi'] = 120

Use `xr.open_dataset` to open the dataset from the remote server.


In [4]:
ifile = 'http://iridl.ldeo.columbia.edu/SOURCES/.NOAA/.NCDC/.ERSST/.version5/.sst/T/%28Jan%201979%29/%28Dec%202018%29/RANGE/X//lon/renameGRID/Y//lat/renameGRID/T//time/renameGRID/time/(days%20since%201979-01-01)/streamgridunitconvert%5Bzlev%5Daverage/dods'
print(ifile)
ds = xr.open_dataset(ifile).load()
print(ds)

http://iridl.ldeo.columbia.edu/SOURCES/.NOAA/.NCDC/.ERSST/.version5/.sst/T/%28Jan%201979%29/%28Dec%202018%29/RANGE/X//lon/renameGRID/Y//lat/renameGRID/T//time/renameGRID/time/(days%20since%201979-01-01)/streamgridunitconvert%5Bzlev%5Daverage/dods
<xarray.Dataset>
Dimensions:  (lat: 89, lon: 180, time: 480)
Coordinates:
  * lat      (lat) float32 -88.0 -86.0 -84.0 -82.0 -80.0 ... 82.0 84.0 86.0 88.0
  * lon      (lon) float32 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * time     (time) datetime64[ns] 1979-01-16T12:00:00 ... 2018-12-16T12:00:00
Data variables:
    sst      (time, lat, lon) float64 nan nan nan nan ... -1.8 -1.8 -1.8 -1.8
Attributes:
    Conventions:  IRIDL


In [ ]:
# # backup plan to load the dataset
# #ds.to_netcdf('ersst.v5.1979-2018.nc', encoding={'sst': {'zlib': True, 'complevel': 1}})
ds = xr.open_dataset('ersst.v5.1979-2018.nc')
ds

In [ ]:
sst = ds.sst
sst

## Explore the data by making simple plots

In [ ]:
sst.isel(time=227).plot()

More experiments on plotting:
* select date/time explicitly
* change colormap/levels
* contourf/contour


In [ ]:
# specify date/time explicitly
# sst.isel(time=227).plot()
sst.sel(time='1997-12-16T12').plot()

In [ ]:
# change levels
sst.sel(time='1997-12-16T12').plot(levels=20)

In [ ]:
# change color map
sst.sel(time='1997-12-16T12').plot(levels=20, center=False, cmap='Spectral_r')

In [ ]:
# change to contourf
sst.sel(time='1997-12-16T12').plot.contourf(levels=20, center=False, cmap='Spectral_r')

## SST change from the first 10 years to the last 10 years

In [ ]:
sst_early = sst.sel(time=slice('1979-01', '1988-12')).mean('time')
sst_late = sst.sel(time=slice('2009-01', '2018-12')).mean('time')
dsst = sst_late - sst_early
dsst.attrs['long_name'] = 'SST change from 1979-1988 to 2009-2018'
dsst

In [ ]:
dsst.plot.contourf(levels=20)
# cooling over the Southern Ocean and Southeast Pacific

Not warming everywhere. Southern Ocean and Southeast Pacific

## Calculate monthly climatology
* multiple-year mean value for each month
* use the `groupby('time.month')` method



In [ ]:
sst_clim = sst.groupby('time.month').mean('time')
sst_clim

In [ ]:
sst_clim.sel(month=1).plot.contourf(levels=20, center=False, cmap='Spectral_r')

In [ ]:
sst_clim.sel(month=7).plot.contourf(levels=20, center=False, cmap='Spectral_r')

Make 12 subplots using a single command.

In [ ]:
sst_clim.plot.contourf(col='month', col_wrap=6, 
                       levels=20, cmap='Spectral_r', center=False)

## Calculate monthly anomaly
Subtract the monthly climatology from the raw SST data.

In [ ]:
ssta = sst.groupby('time.month') - sst_clim
ssta

In [ ]:
ssta.sel(time=slice('1997-12', '1998-02')).mean('time').plot.contourf(levels=20)

## Calculate the Nino3.4 index
* SST averaged over the Nino3.4 region: 170W-120W, 5S-5N
![ENSO](http://www.bom.gov.au/climate/enso/indices/oceanic-indices-map.gif)
http://www.bom.gov.au/climate/enso/indices/oceanic-indices-map.gif

In [ ]:
nino34 = ssta.sel(lon=slice(360-170,360-120),lat=slice(-5,5)).mean(['lon','lat'])
nino34.attrs['long_name'] = 'Nino3.4 index'
nino34

In [ ]:
nino34.plot()
plt.axvline('2015-12', color='gray', ls='--')
plt.text('2015-12', 2, '2015-12', rotation=-45, color='gray', )
plt.axvline('1997-12', color='gray', ls='--')
plt.text('1997-12', 2, '1997-12', rotation=-45, color='gray', )
plt.axvline('1982-12', color='gray', ls='--')
plt.text('1982-12', 2, '1982-12', rotation=-45, color='gray', )

## Seasonality of El Nino/Lanina

In [ ]:
nino34.groupby('time.month').std('time').plot()
plt.ylabel('Nino3.4 standard deviation [$^\circ$C]')

December shows the largest variability!